# APIM ❤️ AI Agents

## Realtime Audio with MCP tools and Agents lab
![flow](../../images/realtime-mcp-agents.gif)

Playground to experiment the with [Azure OpenAI Realtime API](https://learn.microsoft.com/en-us/azure/ai-services/openai/realtime-audio-reference) for text and audio with integrations via [Model Context Protocol](https://modelcontextprotocol.io/) with Azure API Management to enable plug & play of tools to LLMs. Leverages the [credential manager](https://learn.microsoft.com/en-us/azure/api-management/credentials-overview) for  managing OAuth 2.0 tokens to backend tools and [client token validation](https://learn.microsoft.com/en-us/azure/api-management/validate-jwt-policy) to ensure end-to-end authentication and authorization.   
This lab includes the following MCP servers:
- Basic weather service: provide tools to get cities for a given country and retrieve random weather information for a specified city.
- Spotify service: provide tools to authenticate on Spotify using the APIM Credential Manager, retrieves Playlists, Latest releases, start/stop music playing, and more. 
- ServiceNow incidents MCP Server: provides tools to authenticates on ServiceNow using the APIM Credential Manager, lists incidents, retrieves a particular incident and create a new one.

### Result
![result](result.png)

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
%pip install -r requirements.txt

In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "uksouth"

aiservices_config = [{"name": "foundry1", "location": "swedencentral"}]

models_config = [{"name": "gpt-realtime", "publisher": "OpenAI", "version": "2025-08-28", "sku": "GlobalStandard", "capacity": 10}]

apim_sku = 'Basicv2'
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

inference_api_path = 'inference'  # path to the inference API in the APIM service
inference_api_type = "websocket"  
inference_api_version = "2024-10-01-preview"
foundry_project_name = deployment_name

build = 0
weather_mcp_server_image = "weather-mcp-server"
weather_mcp_server_src = "src/weather/mcp-server"

spotify_mcp_server_image = "spotify-mcp-server"
spotify_mcp_server_src = "src/spotify/mcp-server"

utils.print_ok('Notebook initialized')

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "aiServicesConfig": { "value": aiservices_config },
        "modelsConfig": { "value": models_config },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "inferenceAPIPath": { "value": inference_api_path },
        "inferenceAPIType": { "value": inference_api_type },
        "foundryProjectName": { "value": foundry_project_name }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    log_analytics_id = utils.get_deployment_output(output, 'logAnalyticsWorkspaceId', 'Log Analytics Id')
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    for subscription in apim_subscriptions:
        subscription_name = subscription['name']
        subscription_key = subscription['key']
        utils.print_info(f"Subscription Name: {subscription_name}")
        utils.print_info(f"Subscription Key: ****{subscription_key[-4:]}")
    api_key = apim_subscriptions[0].get("key") # default api key to the first subscription key
    container_registry_name = utils.get_deployment_output(output, 'containerRegistryName', 'Container Registry Name')
    weather_containerapp_resource_name = utils.get_deployment_output(output, 'weatherMCPServerContainerAppResourceName', 'Weather Container App Resource Name')
    spotify_containerapp_resource_name = utils.get_deployment_output(output, 'spotifyMCPServerContainerAppResourceName', 'Spotify Container App Resource Name')


<a id='4'></a>
### 4️⃣ Build and deploy the MCP Servers



In [ ]:
build = build + 1 # increment the build number

utils.run(f"az acr build --image {weather_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {weather_mcp_server_src}/Dockerfile {weather_mcp_server_src}/. --no-logs", 
         "Weather MCP Server image was successfully built", "Failed to build the Weather MCP Server image")
utils.run(f'az containerapp update -n {weather_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{weather_mcp_server_image}:v0.{build}"', 
         "Weather MCP Server deployment succeeded", "Weather MCP Server deployment failed")

utils.run(f"az acr build --image {spotify_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {spotify_mcp_server_src}/Dockerfile {spotify_mcp_server_src}/. --no-logs", 
          "Spotify MCP Server image was successfully built", "Failed to build the Spotify MCP Server image")
utils.run(f'az containerapp update -n {spotify_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{spotify_mcp_server_image}:v0.{build}"', 
          "Spotify MCP Server deployment succeeded", "Spotify MCP Server deployment failed")


<a id='testconnection'></a>
### 🧪 Test the connection to the MCP servers and List Tools



In [ ]:
import os, json, asyncio, time, requests
from mcp import ClientSession
from mcp.client.sse import sse_client
import nest_asyncio
nest_asyncio.apply()

async def list_tools(server_url, authorization_header = None):
    headers = {"Authorization": authorization_header} if authorization_header else None
    async with sse_client(server_url, headers) as streams:
        async with ClientSession(streams[0], streams[1]) as session:
            await session.initialize()

            response = await session.list_tools()
            tools = response.tools
    print(f"✅ Connected to server {server_url}")
    print("⚙️ Tools:")
    for tool in tools:
        print(f"  - {tool.name}")
        print(f"     Input Schema: {tool.inputSchema}")
    
asyncio.run(list_tools(f"{apim_resource_gateway_url}/spotify/mcp/sse"))
asyncio.run(list_tools(f"{apim_resource_gateway_url}/weather/sse"))


<a id='inspector'></a>
### 🧪 (optional) Use the [MCP Inspector](https://modelcontextprotocol.io/docs/tools/inspector) for testing and debugging the MCP servers

#### Execute the following steps:
1. Execute `npx @modelcontextprotocol/inspector` in a terminal
2. Open the provided URL in a browser
3. Set the transport type as SSE
4. Provide the MCP server url and click connect
5. Select the "Tools" tab to see and run the available tools

<a id='sk'></a>
### 🧪 Execute a Realtime [Semantic Kernel Agent using MCP Tools](https://devblogs.microsoft.com/semantic-kernel/integrating-model-context-protocol-tools-with-semantic-kernel-a-step-by-step-guide/) via Azure API Management

👉 Use [this samples](https://github.com/microsoft/semantic-kernel/tree/main/python/samples/concepts/realtime) for more advanced use cases


In [ ]:
from semantic_kernel.connectors.ai.open_ai import (
    AzureRealtimeWebsocket,
    AzureRealtimeExecutionSettings,
    ListenEvents,
    TurnDetection
)
from semantic_kernel.contents import RealtimeTextEvent
from semantic_kernel.contents.text_content import TextContent
from semantic_kernel.connectors.mcp import MCPSsePlugin

realtime_agent = AzureRealtimeWebsocket(
            endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
            deployment_name=models_config[0]['name'],
            azure_openai_realtime_deployment_name=models_config[0]['name'],
            api_key=api_key,
            api_version=inference_api_version)

weather_plugin = MCPSsePlugin(
    name="Weather",
    url=f"{apim_resource_gateway_url}/weather/sse",
    description="Weather Plugin",
)
    
settings = AzureRealtimeExecutionSettings(modalities=["text"], turn_detection=TurnDetection(type="server_vad", create_response=True, silence_duration_ms=800, threshold=0.8),

                                          instructions="You are a helpful assistant that provides information about the weather, using just the text modality",)
async with realtime_agent(settings=settings, create_response=True, plugins=[weather_plugin]) as connection:
    await connection.send(RealtimeTextEvent(text=TextContent(text="What's the weather like in Lisbon?")))

    async for event in connection.receive():
        if event.service_type == "response.text.done":
            print(event, flush=True, end="")
        elif event.service_type == "response.done":
            break


<a id='spotifyconfig'></a>
### 5️⃣ Create a Spotify OAuth app and configure the credential provider

##### Step 1 - [Create an account on Spotify Developer portal](https://developer.spotify.com/) 
##### Step 2 - [Create an app with redirect URI from the next step](https://developer.spotify.com/dashboard)

👉 Use the Authorization callback URL that is provided below  
👉 Copy the Client ID and Client secret

##### Step 3 - Configure the credential provider in API Management

👉 You just need to update the Client ID and Client secret on the existing `spotify` credential manager provider  

In [ ]:
print(f"Authorization callback URL: https://authorization-manager.consent.azure-apim.net/redirect/apim/{apim_resource_name}")

<a id='rtfunctioncalling'></a>
### 🧪 Test the Realtime API using FastRTC + Gradio
FastRTC is an elegant realtime library communication library to enable you to easily and quickly build RTC application both using websockets and WebRTC.

Please ensure you have run the pip command succefully to install all required packages

👉 Tip: Restart the Python Kernel to stop the FastRTC server 


In [ ]:
import asyncio, base64, json, random, openai
import gradio as gr
import numpy as np
from fastrtc import (
    AdditionalOutputs,
    AsyncStreamHandler,
    Stream,
    wait_for_item,
    UIArgs
)
from gradio.utils import get_space
from mcp_client import OAI_RT_SSEMCPClient

# Define the sample rate (Hz)
SAMPLE_RATE = 24000

AZURE_OPENAI_API_ENDPOINT = f"{apim_resource_gateway_url}/{inference_api_path}"
AZURE_OPENAI_API_KEY = api_key
AZURE_OPENAI_API_VERSION = inference_api_version
AZURE_OPENAI_DEPLOYMENT_NAME = models_config[0]['name']

class OpenAIHandler(AsyncStreamHandler):
    mcp_config = {
        'spotify': f"{apim_resource_gateway_url}/spotify/mcp/sse",
        'weather': f"{apim_resource_gateway_url}/weather/sse"
        }
    mcp_servers = {}
    tool_server_map = {}

    def __init__(self) -> None:
        super().__init__(
            expected_layout="mono",
            output_sample_rate=SAMPLE_RATE,
            output_frame_size=480,  # In this example we choose 480 samples per frame.
            input_sample_rate=SAMPLE_RATE,
        )
        self.connection = None
        if len(self.mcp_config) > 0:
            print ("⚙️ Initializing all MCPs")
            for name, url in self.mcp_config.items():
                self.mcp_servers[name] = OAI_RT_SSEMCPClient(server_name=name, url=url)
        else:
            print ("no MCPs")
        self.output_queue = asyncio.Queue()

    async def start_up(self):
        """
        Establish a persistent realtime connection to the Azure OpenAI backend.
        The connection is configured for server‐side Voice Activity Detection.
        """
        self.client = openai.AsyncAzureOpenAI(
            azure_endpoint=AZURE_OPENAI_API_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION,
        )
        for name, server in self.mcp_servers.items():
                await server.start()
        # When using Azure OpenAI realtime (beta), set the model/deployment identifier
        async with self.client.realtime.connect(
            model=AZURE_OPENAI_DEPLOYMENT_NAME  # Replace with your deployed realtime model id on Azure OpenAI.
        ) as conn:
            # Configure the session to use server-based voice activity detection (VAD)
            session_config = await self.session_config()
            await conn.session.update(session=session_config) # type: ignore
            self.connection = conn

            # Uncomment the following line to send a welcome message to the assistant.
            # await self.welcome()

            async for event in self.connection:
                # Handle interruptions
                if event.type == "input_audio_buffer.speech_started":
                    self.clear_queue()
                if event.type == "conversation.item.input_audio_transcription.completed":
                    # This event signals that an input audio transcription is completed.
                    await self.output_queue.put(AdditionalOutputs(event))
                if event.type == "response.audio_transcript.done":
                    # This event signals that a response audio transcription is completed.
                    await self.output_queue.put(AdditionalOutputs(event))
                if event.type == "response.function_call_arguments.done":
                    await self.get_tool_response(event)
                if event.type == "conversation.item.created":
                    await self.output_queue.put(AdditionalOutputs(event))
                if event.type == "response.audio.delta":
                    # For incremental audio output events, decode the delta.
                    await self.output_queue.put(
                        (
                            self.output_sample_rate,
                            np.frombuffer(base64.b64decode(event.delta), dtype=np.int16).reshape(1, -1),
                        ),
                    )

    def copy(self):
        return OpenAIHandler()

    async def welcome(self):
        await self.connection.conversation.item.create( # type: ignore
            item={
                "type": "message",
                "role": "user",
                "content": [{"type": "input_text", "text": "what's your name?"}],
            }
        )
        await self.connection.response.create() # type: ignore

    async def receive(self, frame: tuple[int, np.ndarray]) -> None:
        """
        Receives an audio frame from the stream and sends it into the realtime API.
        The audio data is encoded as Base64 before appending to the connection's input.
        """
        if not self.connection:
            return
        _, array = frame
        array = array.squeeze()
        # Encode audio as Base64 string
        audio_message = base64.b64encode(array.tobytes()).decode("utf-8")
        await self.connection.input_audio_buffer.append(audio=audio_message)  # type: ignore

    async def emit(self) -> tuple[int, np.ndarray] | AdditionalOutputs | None:
        """
        Waits for and returns the next output from the output queue.
        The output may be an audio chunk or an additional output such as transcription.
        """
        return await wait_for_item(self.output_queue)

    async def shutdown(self) -> None: # type: ignore
        # await self.weather_mcp.stop()
        if self.connection:
            await self.connection.close()
            self.connection = None

    async def session_config(self):
        """Returns a random value from the predefined list."""
        values = ['alloy', 'ash', 'ballad', 'coral', 'echo', 'sage', 'shimmer', 'verse']
        tools = []
        ### Get all tools from all active servers
        for name, server in self.mcp_servers.items():
                current_server_tools = await server.list_tools()
                for tool in current_server_tools:
                    self.tool_server_map[tool['name']]= name
                tools = tools + current_server_tools
        print(self.tool_server_map)
        voice = random.choice(values)
        print(voice)
        ### for details on available param: https://platform.openai.com/docs/api-reference/realtime-sessions/create
        SESSION_CONFIG={
            "input_audio_transcription": {
                "model": "whisper-1",
            },
            "turn_detection": {
                "threshold": 0.4,
                "silence_duration_ms": 600,
                "type": "server_vad"
            },
            "instructions": f"""You are a DJ named {voice}! 
                                You are a helpful, calm and cheerful agent who responds with a clam accent, but also can speak in any language or accent. 
                                If you need to call a spotify tool, inform the user that you need first to call the authorization tool.
                                After calling the authorization tool, send a text event with the link URL and don't read the link.
                                After sending the event with the link, inform the user that he just needs to click on the provide link to get authorized on Spotify.""",
            "voice": voice,
            "modalities": ["text", "audio"], ## required to solicit the initial welcome message
            "tools": tools
        }
        return SESSION_CONFIG

    async def get_tool_response(self, event):
        ### findout which MCP server to call
        server_name = self.tool_server_map[event.name]
        ### call the target
        response = await self.mcp_servers[server_name].call_tool(tool_call=event.model_dump())
        await self.connection.conversation.item.create(item=response) # type: ignore
        await self.connection.response.create() # type: ignore

def on_open(ws):
    print("Connected to server.")

def on_message(ws, message):
    data = json.loads(message)
    print("Received event:", json.dumps(data, indent=2))

def update_chatbot(chatbot: list[dict], event):
    """
    Append the completed transcription to the chatbot messages.
    """
    if event.type == "conversation.item.input_audio_transcription.completed":
        chatbot.append({"role": "user", "content": event.transcript})
    elif event.type == "response.audio_transcript.done":
        chatbot.append({"role": "assistant", "content": event.transcript})
    elif event.type == "conversation.item.created":
        if event.item and event.item.type == "function_call_output":
            output = str(event.item.output)
            expected_output = "Please authorize by opening this link:" # this string must match what was coded in the mcp server
            if expected_output in output:
                link = f'<a href="{output.replace(expected_output, "").strip()}">👉 Click here to authorize</a>'
                chatbot.append({"role": "assistant", "content": link})
    return chatbot

ui_args: UIArgs = UIArgs(
    title="APIM ❤️ MCP - DJ Contoso Assistant 🎧",
)
chatbot = gr.Chatbot(type="messages")
latest_message = gr.Textbox(type="text", visible=True)

# Instantiate the Stream object that uses the OpenAIHandler.
stream = Stream(
    OpenAIHandler(),
    mode="send-receive",
    modality="audio",
    additional_inputs=[chatbot],
    additional_outputs=[chatbot],
    additional_outputs_handler=update_chatbot,
    ui_args=ui_args,
)

if __name__ == "__main__":
    stream.ui.launch(server_port=7897)


<a id='servicenowconfig'></a>
### 6️⃣ Create a ServiceNow OAuth app and configure the credential provider

ℹ️ If you do not wish to use ServiceNow, please skip these steps

#### Step 1 - [Register the application in ServiceNow](https://www.servicenow.com/docs/bundle/yokohama-application-development/page/build/pipelines-and-deployments/task/create-oauth-api-endpoints-for-external-clients.html)

👉 Use the Authorization callback URL that is provided bellow  
👉 Copy the Client ID and Client secret

#### Step 2 - Configure the credential provider in API Management

👉 You just need to update the Client ID and Client secret on the existing `servicenonw` credential manager provider  

In [ ]:
print(f"Authorization callback URL: https://authorization-manager.consent.azure-apim.net/redirect/apim/{apim_resource_name}")

<a id='servicenowtest'></a>
### 🧪 Run the ServiceNow MCP Server with VS Code to manage ServiceNow incidents

1. [Configure the ServiceNow MCP Server in VS Code](https://code.visualstudio.com/docs/copilot/chat/mcp-servers#_add-an-mcp-server) 
2. Type in the chat the following prompt: `Please list my servicenow incidents`
3. The agent will suggest running the `authorize_servicenow` tool.
4. Once the user accepts to run the tool, the agent will call the `authorize_servicenow` and provide an URL to proceed with the authentication and authorization on ServiceNow.
5. After the user confirms that it's done, the agent will suggest running the `list_incidents` tool.
6. Once the user accepts to run the `list_incidents` tool, the agent will provide the list of incidents for the connected ServiceNow instance.
7. You can also retrieve details for a specific incident or create a new one.

✨ Type in the chat the following prompt: `Create a ServiceNow incident for each GitHub issue`. To combine GitHub and ServiceNow MCP Servers.

<a id='validate-jwt'></a>
### 🔐 (Optional) Implement [authorization policies](src/github/apim-api/auth-client-policy.xml) on MCP endpoints

👉 To ensure the enforcement of valid security tokens, we apply the `validate-jwt` policy to the `/sse` and `/messages` endpoints. The following code snippet demonstrates the application of this policy to GitHub API operations for token validation:

In [ ]:
policy_xml_file = "src/github/apim-api/auth-client-policy.xml"

with open(policy_xml_file, 'r') as file:
    policy_xml = file.read()
    utils.update_api_operation_policy(subscription_id, resource_group_name, apim_resource_name, "github-mcp", "sse", policy_xml)
    utils.update_api_operation_policy(subscription_id, resource_group_name, apim_resource_name, "github-mcp", "messages", policy_xml)

<a id='unauthorizedtest'></a>
### 🧪 Test the authorization **WITHOUT** a valid token

In [ ]:
# Unauthenticated call should fail with 401 Unauthorized
import requests
utils.print_info("Calling sse endpoint WITHOUT authorization...")
response = requests.get(f"{apim_resource_gateway_url}/github/sse", headers={"Content-Type": "application/json"})
if response.status_code == 401:
    utils.print_ok("Received 401 Unauthorized as expected")
elif response.status_code == 200:
    utils.print_error("Call succeeded. Double check that validate-jwt policy has been deployed to sse endpoint")
else:
    utils.print_error(f"Unexpected status code: {response.status_code}")


<a id='authorizedtest'></a>
### 🧪 Test the authorization **WITH** a valid token

In [ ]:
import requests
# Authenticated call should succeed
utils.print_info("Calling sse endpoint WITH authorization...")
output = utils.run("az account get-access-token --resource \"https://azure-api.net/authorization-manager\"")
if output.success and output.json_data:
    access_token = output.json_data.get('accessToken')
    response = requests.get(f"{apim_resource_gateway_url}/github/sse", stream=True,
                            headers={"Content-Type": "application/json", "Authorization": "Bearer " + str(access_token)})
    if response.status_code == 200:
        utils.print_ok("Received status code 200 as expected")
    else:
        utils.print_error(f"Unexpected status code: {response.status_code}")



<a id='kql'></a>
### 🔍 Display model usage


In [ ]:
import pandas as pd

query = "model_usage"

output = utils.run(f"az monitor log-analytics query -w {log_analytics_id} --analytics-query \"{query}\"", "Retrieved log analytics query output", "Failed to retrieve log analytics query output") 
if output.success and output.json_data:
    table = output.json_data
    display(pd.DataFrame(table))


<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.